# Исследование надежности заемщиков

## 1. Обзор данных

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("/datasets/data.csv")

In [3]:
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблцице 12 столбцов, типы данных int64, object, float64 

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

Также необходимо изменить тип данных в столбцах: 
- days_employed (подсчет стажа идет кратно дню, поэтому должно быть целое число, а не вещественное. 
- total_income (ежемесячный доход лучше также записать как целые числа для удобства расчетов)

Предварительно можно утверждать, что данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в колоноках — разный регистр и не верный тип данных.
Чтобы двигаться дальше, нужно устранить проблемы в данных.

## 2. Удаление дубликатов.

In [5]:
data.duplicated().sum()

54

In [6]:
data = data.drop_duplicates().reset_index(drop=True)

## 3. Заполнение пропусков

In [7]:
data.isna().sum()

children               0
days_employed       2120
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2120
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения: days_employed и total_income. Необходимо заполнить эти пробелы, а также изменить тип данных на int.

В столбце 'days_employed' пропуски не важны для работы. Достаточно заменить их медианными значениями.

Ежемесячный доход имеет слишком большой разрыв и среднее значение может исказить данные, поэтому опять прибегну к медианному значению.

In [8]:
data['days_employed'] = data.groupby('dob_years')['days_employed'].transform(lambda x : x.fillna(x.median()))
data.groupby('dob_years')['days_employed'].median()
data.loc[data['days_employed'] == 0, 'days_employed'] = data.groupby('dob_years')['days_employed'].transform('median')

In [9]:
data['total_income'] = data.groupby('income_type')['total_income'].transform(lambda x : x.fillna(x.median()))
data.groupby('income_type')['total_income'].median()
data.loc[data['total_income'] == 0, 'total_income'] = data.groupby('income_type')['total_income'].transform('median')

In [10]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

## 4. Изменение типов данных.

Изменяю тип данных в столбце 'total_income'. А в 'days_employed' уберу минусы. 

In [11]:
data['total_income'] = data[['total_income','days_employed']].astype('int')

In [12]:
data['days_employed']=data['days_employed'].abs()

In [13]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Исправляю аномальные данные в столбце. 

In [14]:
data['children'] = data['children'].replace(20,2)

In [15]:
data['children'] = data['children'].replace(-1,1)

## 5. Поиск и удаление неявных дубликатов.

Вывожу на экран список уникальных названий столбца 'education', отсортированный в алфавитном порядке.

In [16]:
sorted(data['education'].unique())

['ВЫСШЕЕ',
 'Высшее',
 'НАЧАЛЬНОЕ',
 'НЕОКОНЧЕННОЕ ВЫСШЕЕ',
 'Начальное',
 'Неоконченное высшее',
 'СРЕДНЕЕ',
 'Среднее',
 'УЧЕНАЯ СТЕПЕНЬ',
 'Ученая степень',
 'высшее',
 'начальное',
 'неоконченное высшее',
 'среднее',
 'ученая степень']

В столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Нужно привести их к одному регистру. 

In [17]:
data['education'] = data['education'].str.lower()

In [18]:
sorted(data['education'].unique())

['высшее', 'начальное', 'неоконченное высшее', 'среднее', 'ученая степень']

## 6. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создаю новый датафрейм со столбцами education и education_id

In [19]:
data_pivot_education = data.pivot_table(index=['education','education_id'],values='debt',aggfunc=['count'])

In [20]:
display(data_pivot_education)

,,count
,,debt
education,education_id,
высшее,0,5251
начальное,3,282
неоконченное высшее,2,744
среднее,1,15188
ученая степень,4,6


Создаю новый датафрейм со столбцами family_status и family_status_id

In [21]:
data_pivot_family_status = data.pivot_table(index=['family_status','family_status_id'],values='debt',aggfunc=['count'])

In [22]:
display(data_pivot_family_status)

,,count
,,debt
family_status,family_status_id,
Не женат / не замужем,4,2810
в разводе,3,1195
вдовец / вдова,2,959
гражданский брак,1,4163
женат / замужем,0,12344


Удаляю из исходного датафрейма столбцы education и family_status

In [23]:
data = data.drop(columns=['education','family_status'])


In [24]:
data.head(15)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926.185831,27,0,1,M,компаньон,0,255763,покупка жилья
6,0,2879.202052,43,0,0,F,компаньон,0,240525,операции с жильем
7,0,152.779569,50,1,0,M,сотрудник,0,135823,образование
8,2,6929.865299,35,0,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188.756445,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи


## 7. Категоризация дохода.

На основании диапазонов указанных ниже, создаю столбец total_income_category с категориями:
- 0-30000 - "Е"
- 30001-50000 - "D"
- 50001 - 200000 - "С"
- 200001 - 1000000 - "В"
- 1000001 и выше - "А"

In [25]:
def total_income_category(total_income):
    if total_income<30000:
        return 'E'
    if 30001<total_income<50000:
        return 'D'
    if 50001<total_income<200000:
        return 'C'
    if 200001<total_income<1000000:
        return 'B'
    return 'A'

In [26]:
data['total_income_category'] = data['total_income'].apply(total_income_category)

## 8. Категоризация целей кредита.

Создаю функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:
- ‘операции с автомобилем’,
- ‘операции с недвижимостью’,
- ‘проведение свадьбы’,
- ‘получение образования’.

In [27]:
def purpose_category(purpose):
    if purpose == 'автомобили' or purpose =='автомобиль' or purpose =='на покупку автомобиля' or purpose =='на покупку подержанного автомобиля' or purpose =='на покупку своего автомобиля' or purpose =='приобретение автомобиля' or purpose =='свой автомобиль' or purpose =='сделка с автомобилем' or purpose =='сделка с подержанным автомобилем':
        return 'операции с автомобилем'
    if purpose == 'высшее образование' or purpose =='дополнительное образование' or purpose =='заняться высшим образованием' or purpose =='заняться образованием' or purpose =='образование' or purpose =='получение высшего образования' or purpose =='получение дополнительного образования' or purpose =='получение образования' or purpose =='профильное образование':
        return 'получение образования'
    if purpose == 'на проведение свадьбы' or purpose =='свадьба' or purpose =='сыграть свадьбу':
        return 'проведение свадьбы'
    return 'операции с недвижимостью'

In [28]:
data['purpose_category'] = data['purpose'].apply(purpose_category)

In [29]:
data

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
...,...,...,...,...,...,...,...,...,...,...,...,...
21466,1,4529.316663,43,1,1,F,компаньон,0,224791,операции с жильем,B,операции с недвижимостью
21467,0,343937.404131,67,1,0,F,пенсионер,0,155999,сделка с автомобилем,C,операции с автомобилем
21468,1,2113.346888,38,1,1,M,сотрудник,1,89672,недвижимость,C,операции с недвижимостью
21469,3,3112.481705,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,B,операции с автомобилем


## Ответы на вопросы.

### 1. Есть ли зависимость между количеством детей и возвратом кредита?

In [30]:
data.groupby('children').agg(число_клиентов=('debt','count'), число_должников=('debt','sum'), доля_должников=('debt','mean')).\
                            style.format({'доля_должников' : '{:.2%}'.format})

,число_клиентов,число_должников,доля_должников
children,,,
0,14107,1063,7.54%
1,4856,445,9.16%
2,2128,202,9.49%
3,330,27,8.18%
4,41,4,9.76%
5,9,0,0.00%


### Вывод:
Зависимость есть. При отсутсвии детей вероятность возврата кредита выше. 

### 2. Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [31]:
data.groupby('family_status_id').agg(число_клиентов=('debt','count'), число_должников=('debt','sum'), доля_должников=('debt','mean')).\
                            style.format({'доля_должников' : '{:.2%}'.format})

,число_клиентов,число_должников,доля_должников
family_status_id,,,
0,12344,931,7.54%
1,4163,388,9.32%
2,959,63,6.57%
3,1195,85,7.11%
4,2810,274,9.75%


In [32]:
display(data_pivot_family_status)

,,count
,,debt
family_status,family_status_id,
Не женат / не замужем,4,2810
в разводе,3,1195
вдовец / вдова,2,959
гражданский брак,1,4163
женат / замужем,0,12344


### Вывод:
Вдовец/вдова с наибольшей вероятностью вернут кредит в срок. У тех кто в разводе вероятность возврата кредита в срок ниже. Еще ниже вероятность возврата кредита в срок у женатых и замужних, и хуже всего возвращают кредит в срок те кто не замужем и не женат. 

### 3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?


In [33]:
data.groupby('total_income_category').agg(число_клиентов=('debt','count'), число_должников=('debt','sum'), доля_должников=('debt','mean')).\
                            style.format({'доля_должников' : '{:.2%}'.format})

,число_клиентов,число_должников,доля_должников
total_income_category,,,
A,26,2,7.69%
B,5041,356,7.06%
C,16032,1360,8.48%
D,350,21,6.00%
E,22,2,9.09%


### Вывод:
Люди с уровнем дохода от 30001 до 50000 вероятнее вернут кредит в срок. Также люди с зарабоком от 50001 до 200000 вероятнее вернут кредит в срок. Меньше вероятности что вернут кредит в срок люди с заработком от 200001 до 1000000. Хуже с выплатой кредита в срок у людей с заработком от 1000001 и выше. А люди сдоходом до 30000 вероятнее не смогут вернуть кредит в срок.

### 4.Как разные цели кредита влияют на его возврат в срок?

In [34]:
data.groupby('purpose_category').agg(число_клиентов=('debt','count'), число_должников=('debt','sum'), доля_должников=('debt','mean')).\
                            style.format({'доля_должников' : '{:.2%}'.format})

,число_клиентов,число_должников,доля_должников
purpose_category,,,
операции с автомобилем,4308,403,9.35%
операции с недвижимостью,10814,782,7.23%
получение образования,4014,370,9.22%
проведение свадьбы,2335,186,7.97%


### Вывод:
При операциях с недвижимостью и с проведением свадьбы вероятность возврата кредита в срок возрастает, а вот при операциях с автомобилем и получением образования вероятность возврата кредита в срок падает. 

## Общий вывод:

Из проведенного анализа следует, что количество детей влияет на погашение кредита в срок. Категория клиентов без детей вероятнее возвращает кредит в срок. Что касается семейного положения, то самыми надежными оказались категории:
- вдовец/вдова
- в разводе

Хуже всего с возвратом в срок обстоят дела с незамужними и неженатыми. 

Также уровень дохода влияет на выплату в срок. Самыми надежными оказались категории с уровнем дохода от 30001 до 200000. Все, кто получает меньше или больше данного диапазона заработка с большей вероятность не вернут кредит в срок. 

На возврат кредита в срок также влияет и цель кредита. При операциях с недвижимостью и проведением свадьбы вероятность погашения в срок возростает. А вот операции с автомобилем самые рискованные.
